# Part 9

Universal code that we will use for the entire notebook.

In [1]:
# Import necessary libraries 
from openai import OpenAI  # Used for interacting with OpenAI's API
from typing_extensions import override  # Used for overriding methods in subclasses
from openai import AssistantEventHandler # Used for handling events related to OpenAI assistants

# Additional libraries for time and date manipulation
import time
import pytz
import datetime 

In [2]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

## Creating Assistants, Threads, and Messages Review
Let's create a new assistant, thread, and some messages for us to use later on and to review the code for creating them.

### Creating an Assistant
First, let's make an Assistant we can use to communicate with our run.

In [3]:

# Create an assistant using the client library.
assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    instructions="You are a helpful assistant.",  # Set the instructions for the assistant.
    name="Son of Run Tester Assistant",  # Give the assistant a name.
    metadata={  # Add metadata about the assistant's capabilities.
        "holds_threads": "True",
        "likes_threads": "True",
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

# Print the details of the created assistant to check its properties.
print(assistant)  # Print the full assistant object.
print("\n\n")
print(assistant.name)  # Print the name of the assistant.
print(assistant.metadata)  # Print the metadata of the assistant.


Assistant(id='asst_E4ge1Ccfdxa6dqy2m2R0GUMl', created_at=1715885985, description=None, instructions='You are a helpful assistant.', metadata={'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}, model='gpt-4o', name='Son of Run Tester Assistant', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Son of Run Tester Assistant
{'holds_threads': 'True', 'likes_threads': 'True', 'holds_messages': 'True', 'likes_messages': 'True'}


### Creating a Thread
Now, let's create a Thread that can be used to hold our messages.

In [5]:
# Create a thread using the OpenAI API and store it in a variable.
# The metadata specifies a user identifier.
thread = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console.
print(thread)


Thread(id='thread_Zupwc2SHMBiiefHIPiEWLPOt', created_at=1715886025, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Creating a Message
Finally, let's create a Message that we can go into the Thread for use later.

In [27]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="Tell me what a penguin is in 100 words or less.",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message

Message(id='msg_XosutquZPKkgU3ie08NBVpMw', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')], created_at=1715886947, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Zupwc2SHMBiiefHIPiEWLPOt')


msg_XosutquZPKkgU3ie08NBVpMw
[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')]
Tell me what a penguin is in 100 words or less.
user


## Creating Streaming Run
Let's create a run and get some output!

In [7]:
# Create an EventHandler class to define how we want to handle the events in the response stream.
# Normally, you would define this class at the top of your script or in a separate file.

class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    @override
    def on_text_created(self, text) -> None:
        """Handle the event when text is first created.
        
        This method is called when a complete piece of text is generated by the assistant.
        """
        # Print a prompt indicating the start of assistant's text response.
        # 'flush=True' ensures that the output is immediately written to the console.
        print("\nassistant text > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text).
        
        This method is called for incremental text updates, useful for streaming responses.
        """
        # Print the incremental text as it is being generated.
        # 'end=""' prevents adding a new line after each delta.
        # 'flush=True' ensures that the output is immediately written to the console.
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        """Handle the event when a tool call is created.
        
        This method is called when the assistant makes a call to an external tool.
        """
        # Print a prompt indicating the assistant is using a tool.
        # 'flush=True' ensures that the output is immediately written to the console.
        print(f"\nassistant tool > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        """Handle the event when there is a delta (update) in a tool call.
        
        This method is called for incremental updates during the tool call's execution.
        """
        if delta.type == 'code_interpreter':
            # If there is an input for the code interpreter, print it.
            # 'flush=True' ensures that the output is immediately written to the console.
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            # If there are outputs from the code interpreter, print them.
            if delta.code_interpreter.outputs:
                # 'flush=True' ensures that the output is immediately written to the console.
                print("\n\noutput >", flush=True)
            # Loop through each output and print the logs if present.
            for output in delta.code_interpreter.outputs:
                if output.type == "logs":
                    # 'flush=True' ensures that the output is immediately written to the console.
                    print(f"\n{output.logs}", flush=True)


In [8]:
# Use the `stream` SDK helper with the `EventHandler` class to create the Run
# and stream the response.
with client.beta.threads.runs.stream(
    thread_id=thread.id,         # ID of the thread to run.
    assistant_id=assistant.id,   # ID of the assistant to use.
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > A penguin is a flightless seabird native to the Southern Hemisphere, particularly Antarctica. Recognizable for their black and white plumage and upright stance, penguins are excellent swimmers, using their flipper-like wings to navigate underwater. They predominantly eat fish, squid, and krill. Penguins are highly social animals, often living and breeding in large colonies. Notable species include the Emperor Penguin and the Adélie Penguin. Adaptations like thick blubber and tightly packed feathers help them withstand cold temperatures.

## Token Management
We have two options for contolling the amount of tokens used:

max_prompt_tokens (integer or null)
Optional
The maximum number of prompt tokens that may be used over the course of the run. The run will make a best effort to use only the number of prompt tokens specified, across multiple turns of the run. If the run exceeds the number of prompt tokens specified, the run will end with status incomplete. 

max_completion_tokens (integer or null)
Optional
The maximum number of completion tokens that may be used over the course of the run. The run will make a best effort to use only the number of completion tokens specified, across multiple turns of the run. If the run exceeds the number of completion tokens specified, the run will end with status incomplete. 

Let's see it in action!

First we start with max_prompt_tokens to limit the input tokens that can be used.

In [9]:
# Let's start by seeing how many messages we have.
messages = client.beta.threads.messages.list(
    thread_id=thread.id  # ID of the thread to list messages from.
)

# Initialize a counter for the messages.
message_count = 0

# Loop through the messages, print the id and content, and count the messages.
for message in messages:
    print(message.id)      # Print the ID of the message.
    print(message.content)  # Print the content of the message.
    print("\n")             # Print a newline for readability.
    message_count += 1      # Increment the message count.

# Print the total number of messages.
print(f"Total number of messages: {message_count}")


msg_BlviKbV4mqYCYmDwIVWqkuzF
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere, particularly Antarctica. Recognizable for their black and white plumage and upright stance, penguins are excellent swimmers, using their flipper-like wings to navigate underwater. They predominantly eat fish, squid, and krill. Penguins are highly social animals, often living and breeding in large colonies. Notable species include the Emperor Penguin and the Adélie Penguin. Adaptations like thick blubber and tightly packed feathers help them withstand cold temperatures.'), type='text')]


msg_PHMYYlUZGb1jEq6vHOFtRcUD
[TextContentBlock(text=Text(annotations=[], value='Tell me what a penguin is in 100 words or less.'), type='text')]


Total number of messages: 2


In [10]:
# First, let's list our runs to see what we have.
runs = client.beta.threads.runs.list(
    thread_id=thread.id  # ID of the thread to list runs from.
)

# Dump all the runs.
print(runs)
print("\n\n")

# Loop through the runs and print the id and status.
for run in runs:
    print(run.id)      # Print the ID of the run.
    print(run.status)  # Print the status of the run.
    print("\n")        # Print a newline for readability.


SyncCursorPage[Run](data=[Run(id='run_Q2ULPO8WDnBcWtXupEgItkRo', assistant_id='asst_E4ge1Ccfdxa6dqy2m2R0GUMl', cancelled_at=None, completed_at=1715886095, created_at=1715886091, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=1715886092, status='completed', thread_id='thread_Zupwc2SHMBiiefHIPiEWLPOt', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=104, prompt_tokens=44, total_tokens=148), temperature=1.0, top_p=1.0, tool_resources={})], object='list', first_id='run_Q2ULPO8WDnBcWtXupEgItkRo', last_id='run_Q2ULPO8WDnBcWtXupEgItkRo', has_more=False)



run_Q2ULPO8WDnBcWtXupEgItkRo
completed




In [12]:
# Now let's do a run limiting the prompt tokens.
with client.beta.threads.runs.stream(
    thread_id=thread.id,          # ID of the thread to run.
    assistant_id=assistant.id,    # ID of the assistant to use.
    # max_prompt_tokens=5,        # Limit the maximum number of prompt tokens to get an error
    max_prompt_tokens=256,        # Limit the maximum number of prompt tokens.
    event_handler=EventHandler(), # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > A penguin is a flightless seabird native to the Southern Hemisphere, particularly Antarctica. Recognizable for their black and white plumage and upright stance, penguins are excellent swimmers, using their flipper-like wings to navigate underwater. They predominantly eat fish, squid, and krill. Penguins are highly social animals, often living and breeding in large colonies. Notable species include the Emperor Penguin and the Adélie Penguin. Adaptations like thick blubber and tightly packed feathers help them withstand cold temperatures.

In [14]:
# Let's list our runs again to see what we have.
runs = client.beta.threads.runs.list(
    thread_id=thread.id  # ID of the thread to list runs from.
)

# Dump all the runs.
print(runs)
print("\n\n")

# Loop through the runs and print the id and status.
for run in runs:
    print(run.id)      # Print the ID of the run.
    print(run.status)  # Print the status of the run.
    print(run.incomplete_details)  # Print the incomplete details of the run.
    print("\n")        # Print a newline for readability.


SyncCursorPage[Run](data=[Run(id='run_e8IUd2GtKu9CwNYjQ8QoMbfC', assistant_id='asst_E4ge1Ccfdxa6dqy2m2R0GUMl', cancelled_at=None, completed_at=1715886232, created_at=1715886229, expires_at=None, failed_at=None, incomplete_details=IncompleteDetails(reason='max_prompt_tokens'), instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=256, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=1715886230, status='incomplete', thread_id='thread_Zupwc2SHMBiiefHIPiEWLPOt', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=104, prompt_tokens=150, total_tokens=254), temperature=1.0, top_p=1.0, tool_resources={}), Run(id='run_Q2ULPO8WDnBcWtXupEgItkRo', assistant_id='asst_E4ge1Ccfdxa6dqy2m2R0GUMl', cancelled_at=None, completed_at=1715886095, created_at=1715886091, expires_at=None, failed_at=None, incomplete_detai

And now limiting the completion tokens

In [16]:
# Now let's do a run limiting the completion tokens.
with client.beta.threads.runs.stream(
    thread_id=thread.id,         # ID of the thread to run.
    assistant_id=assistant.id,   # ID of the assistant to use.
    # max_completion_tokens=5,    # Limit the maximum number of completion tokens to get an error
    max_completion_tokens=16,    # Limit the maximum number of completion tokens.
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > A penguin is a flightless seabird native to the Southern Hemisphere,

In [17]:
# Now let's list our runs again
runs = client.beta.threads.runs.list(
    thread_id=thread.id
)


# Loop throught the runs and print the id and status
for run in runs:
    print(run.id)
    print(run.status)
    print(run.incomplete_details)
    print("\n")

run_bEqoGmftiIOAc0e8RnBV5pi2
incomplete
IncompleteDetails(reason='max_completion_tokens')


run_e8IUd2GtKu9CwNYjQ8QoMbfC
incomplete
IncompleteDetails(reason='max_prompt_tokens')


run_Q2ULPO8WDnBcWtXupEgItkRo
completed
None




### Truncation Strategy
We will need to manage having a lot of context being sent via messages and managing the tokens sent each time.


truncation_strategy (object)
Optional
Controls for how a thread will be truncated prior to the run. Use this to control the intial context window of the run.

Properties:
type (string)
Required
The truncation strategy to use for the thread. The default is auto. If set to last_messages, the thread will be truncated to the n most recent messages in the thread. When set to auto, messages in the middle of the thread will be dropped to fit the context length of the model, max_prompt_tokens.

last_messages (integer or null)
Optional
The number of most recent messages from the thread when constructing the context for the run.

In [18]:
# Let's start by seeing how many messages we have.
messages = client.beta.threads.messages.list(
    thread_id=thread.id  # ID of the thread to list messages from.
)

# Initialize a counter for the messages.
message_count = 0

# Loop through the messages, print the id and content, and count the messages.
for message in messages:
    print(message.id)      # Print the ID of the message.
    print(message.content)  # Print the content of the message.
    print("\n")             # Print a newline for readability.
    message_count += 1      # Increment the message count.

# Print the total number of messages.
print(f"Total number of messages: {message_count}")

msg_od5KXjXYUMVbpniKpsRn7hoL
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere,'), type='text')]


msg_OwQe0gbPvmc4ecayRPYel2J8
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere, particularly Antarctica. Recognizable for their black and white plumage and upright stance, penguins are excellent swimmers, using their flipper-like wings to navigate underwater. They predominantly eat fish, squid, and krill. Penguins are highly social animals, often living and breeding in large colonies. Notable species include the Emperor Penguin and the Adélie Penguin. Adaptations like thick blubber and tightly packed feathers help them withstand cold temperatures.'), type='text')]


msg_BlviKbV4mqYCYmDwIVWqkuzF
[TextContentBlock(text=Text(annotations=[], value='A penguin is a flightless seabird native to the Southern Hemisphere, particularly Antarctica. Recognizable for

In [22]:
# Now let's use a truncation strategy on the last set of messages.
with client.beta.threads.runs.stream(
    thread_id=thread.id,           # ID of the thread to run.
    assistant_id=assistant.id,     # ID of the assistant to use.
    max_prompt_tokens=500,        # Limit the maximum number of prompt tokens.
    truncation_strategy={          # Adding a truncation strategy to control the context of the conversation.
        "type": "last_messages",   # Strategy to keep only the most recent messages.
        "last_messages": 4         # Keep the last 5 messages in the context window.
    },
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > Of course, I can help with identifying objects, animals, plants, and many other things from images. Please go ahead and upload the image you'd like assistance with.

In [23]:
# Now let's use a truncation strategy where the system decides what messages to cut.
with client.beta.threads.runs.stream(
    thread_id=thread.id,           # ID of the thread to run.
    assistant_id=assistant.id,     # ID of the assistant to use.
    max_prompt_tokens=1000,        # Limit the maximum number of prompt tokens.
    truncation_strategy={          # Adding a truncation strategy to control the context of the conversation.
        "type": "auto"             # Strategy where the system automatically decides which messages to keep.
    },
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()



assistant text > Sure thing! Please upload the image you'd like help identifying.

In [25]:
# Let's see how many messages we have now
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Initialize a counter for the messages
message_count = 0

# Loop through the messages, print the id and content, and count the messages
for message in messages:
    print(message.id)
    print(message.content)
    print("\n")
    message_count += 1  # Increment the message count

# Print the total number of messages
print(f"Total number of messages: {message_count}")


msg_IxntNSAHDgPtv4Qy9JcrNvTZ
[TextContentBlock(text=Text(annotations=[], value="Sure thing! Please upload the image you'd like help identifying."), type='text')]


msg_Jym53SULJSwAto2MwfNdX5vh
[TextContentBlock(text=Text(annotations=[], value="Of course, I can help with identifying objects, animals, plants, and many other things from images. Please go ahead and upload the image you'd like assistance with."), type='text')]


msg_QYIp6QRDRxQsbOEIT5I9PJTJ
[TextContentBlock(text=Text(annotations=[], value="Of course, I can help you identify a wide range of things in images including objects, plants, animals, products, and more. Please go ahead and provide the image you'd like assistance with."), type='text')]


msg_hboPS3HiLmaDo8GGAO8OvkhC
[TextContentBlock(text=Text(annotations=[], value='particularly found in Antarctica. Penguins are well-adapted to life in the water, with their streamlined bodies and flippers allowing them to swim with great agility. Their black and white plumage provid

In [28]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,           # ID of the thread to run.
    assistant_id=assistant.id,     # ID of the assistant to use.
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    # Process the stream until it is complete.
    stream.until_done()


assistant text > A penguin is a flightless seabird native to the Southern Hemisphere, especially Antarctica. Known for their distinctive black and white feathers and upright posture, penguins are excellent swimmers, using their flipper-like wings to maneuver underwater. They primarily feed on fish, squid, and krill. Penguins are social creatures, often forming large colonies for breeding and molting. Their adaptations, such as thick blubber and dense feathers, allow them to survive in frigid temperatures. Species vary in size and behavior, with the Emperor Penguin and Adélie Penguin among the most notable.

## Controlling Tool Choice and Response Format
In addition to all the other arguments we can modify, we also have tool_choice and response_format:

tool_choice (string or object)

Optional

Controls which (if any) tool is called by the model. none means the model will not call any tools and instead generates a message. auto is the default value and means the model can pick between generating a message or calling one or more tools. required means the model must call one or more tools before responding to the user. Specifying a particular tool like {"type": "file_search"} or {"type": "function", "function": {"name": "my_function"}} forces the model to call that tool.

response_format (string or object)

Optional

Specifies the format that the model must output. Compatible with GPT-4o, GPT-4 Turbo, and all GPT-3.5 Turbo models since gpt-3.5-turbo-1106.

Setting to { "type": "json_object" } enables JSON mode, which guarantees the message the model generates is valid JSON.

Important: when using JSON mode, you must also instruct the model to produce JSON yourself via a system or user message. Without this, the model may generate an unending stream of whitespace until the generation reaches the token limit, resulting in a long-running and seemingly "stuck" request. Also note that the message content may be partially cut off if finish_reason="length", which indicates the generation exceeded max_tokens or the conversation exceeded the max context length.



In [29]:
# Use the `stream` SDK helper with the `EventHandler` class to create the Run
# and stream the response.
with client.beta.threads.runs.stream(
    thread_id=thread.id,  # ID of the thread to run.
    assistant_id=assistant.id,  # ID of the assistant to use.
    additional_instructions="Make sure your output is in JSON format.",  # Instructions for JSON output.
    response_format={"type": "json_object"},  # Specify response format as JSON object.
    tool_choice=None,
    event_handler=EventHandler(),  # Custom event handler for processing events.
) as stream:
    stream.until_done()  # Process the stream until complete.



assistant text > {
  "description": "A penguin is a flightless seabird native to the Southern Hemisphere, especially Antarctica. Known for their distinctive black and white feathers and upright posture, penguins are excellent swimmers, using their flipper-like wings to maneuver underwater. They primarily feed on fish, squid, and krill. Penguins are social creatures, often forming large colonies for breeding and molting. Their adaptations, such as thick blubber and dense feathers, allow them to survive in frigid temperatures. Species vary in size and behavior, with the Emperor Penguin and Adélie Penguin among the most notable."
}

## Listing Runs
thread_id
(string)

Required
The ID of the thread the run belongs to.

limit
(integer)

Optional
Defaults to 20
A limit on the number of objects to be returned. Limit can range between 1 and 100, and the default is 20.

order
(string)

Optional
Defaults to desc
Sort order by the created_at timestamp of the objects. asc for ascending order and desc for descending order.

after
(string)

Optional
A cursor for use in pagination. after is an object ID that defines your place in the list. For instance, if you make a list request and receive 100 objects, ending with obj_foo, your subsequent call can include after=obj_foo in order to fetch the next page of the list.

before
(string)

Optional
A cursor for use in pagination. before is an object ID that defines your place in the list. For instance, if you make a list request and receive 100 objects, ending with obj_foo, your subsequent call can include before=obj_foo in order to fetch the previous page of the list.

In [30]:
# List our runs to see what we have
runs = client.beta.threads.runs.list(
    thread_id=thread.id,
    order="desc",  # Order runs in descending order
)

# Loop through the runs and print the id, status, and incomplete details
for run in runs:
    print(run.id)
    print(run.status)
    print(run.incomplete_details)
    print("\n")



run_lQbiwlbMKN9ooXS6Mea1O5qy
completed
None


run_9ru9gjWzQIr2z2z1OCOSAOvL
completed
None


run_uvHJ8dcAlMyFIbprFC2UbFAv
completed
None


run_JkIJJKGcPtnvNtCwYhB8EbnJ
completed
None


run_L7e1qjVqnu25cD4rLWRGAJEF
completed
None


run_MNOkvlhMc558QtaejIephqJy
completed
None


run_DNixdJ2InmA3LqYVvCoL78Xl
completed
None


run_bEqoGmftiIOAc0e8RnBV5pi2
incomplete
IncompleteDetails(reason='max_completion_tokens')


run_e8IUd2GtKu9CwNYjQ8QoMbfC
incomplete
IncompleteDetails(reason='max_prompt_tokens')


run_Q2ULPO8WDnBcWtXupEgItkRo
completed
None




# Part 10

## Retrieve Runs

thread_id
(string)

Required
The ID of the thread that was run.

run_id
(string)

Required
The ID of the run to retrieve.

In [31]:
# List our runs to see what we have
runs = client.beta.threads.runs.list(
    thread_id=thread.id,
    order="desc",  # Order runs in descending order
)

# Convert the normal SyncCursorPage object we get back to an actual list
runs_list = list(runs)

# Check if there are any runs
if runs_list:
    # Get the run id of the last run (first in the list since it's sorted in descending order)
    last_run_id = runs_list[0].id
    print("Last run ID:", last_run_id)

# Loop through the runs and print the id, status, and incomplete details
for run in runs_list:
    print(run.id)
    print(run.status)
    print(run.incomplete_details)
    print("\n")



Last run ID: run_lQbiwlbMKN9ooXS6Mea1O5qy
run_lQbiwlbMKN9ooXS6Mea1O5qy
completed
None


run_9ru9gjWzQIr2z2z1OCOSAOvL
completed
None


run_uvHJ8dcAlMyFIbprFC2UbFAv
completed
None


run_JkIJJKGcPtnvNtCwYhB8EbnJ
completed
None


run_L7e1qjVqnu25cD4rLWRGAJEF
completed
None


run_MNOkvlhMc558QtaejIephqJy
completed
None


run_DNixdJ2InmA3LqYVvCoL78Xl
completed
None


run_bEqoGmftiIOAc0e8RnBV5pi2
incomplete
IncompleteDetails(reason='max_completion_tokens')


run_e8IUd2GtKu9CwNYjQ8QoMbfC
incomplete
IncompleteDetails(reason='max_prompt_tokens')


run_Q2ULPO8WDnBcWtXupEgItkRo
completed
None




In [32]:
# Retrieve the details of a specific run
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    # Use the ID of the first run in the list that was sorted descending and therefore is the last run to happen
    run_id=runs_list[0].id  
)

# Print the details of the retrieved run
print(run)



Run(id='run_lQbiwlbMKN9ooXS6Mea1O5qy', assistant_id='asst_E4ge1Ccfdxa6dqy2m2R0GUMl', cancelled_at=None, completed_at=1715887055, created_at=1715887052, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant. Make sure your output is in JSON format.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format=AssistantResponseFormat(type='json_object'), started_at=1715887052, status='completed', thread_id='thread_Zupwc2SHMBiiefHIPiEWLPOt', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=123, prompt_tokens=666, total_tokens=789), temperature=1.0, top_p=1.0, tool_resources={})


## Create Thread and Run
We have a helper function that can be used to create a thread and run it in one shot.

assistant_id
(string)

Required

The ID of the assistant to use to execute this run.




In [33]:
# Create and run a thread with a specific message
run = client.beta.threads.create_and_run(
    assistant_id=assistant.id,  # ID of the assistant to use.
    thread={
        "messages": [
            {
                "role": "user", 
                "content": "Explain deep learning to me like I'm a 5 year old. In 50 words or less."
            }
        ]
    }
)

# Print the run details
print(run)
print("\n")

# Print the thread ID of the run
print(run.thread_id)
print("\n")

# Sleep for a few seconds to allow the run to complete
time.sleep(10)

# Retrieve the latest status of the run
run = client.beta.threads.runs.retrieve(
    thread_id=run.thread_id,  # Use the thread ID from the created run.
    run_id=run.id  # Use the run ID from the created run.
)

# Print the run ID and status
print(run.id)
print(run.status)
print("\n")

# Get the messages from the thread, ordered in ascending order
messages = client.beta.threads.messages.list(
    thread_id=run.thread_id,  # Use the thread ID from the run.
    order="asc"  # Order messages in ascending order
)

# Loop through the messages and print the content
for message in messages:
    print(message.content)
    print("\n")


Run(id='run_MiFUmNLCvAhTQ7Z0edWTkFRu', assistant_id='asst_E4ge1Ccfdxa6dqy2m2R0GUMl', cancelled_at=None, completed_at=None, created_at=1715888599, expires_at=1715889199, failed_at=None, incomplete_details=None, instructions='You are a helpful assistant.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_dQrBFqUJrcpxHTrIbbU0jMCa', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


thread_dQrBFqUJrcpxHTrIbbU0jMCa


run_MiFUmNLCvAhTQ7Z0edWTkFRu
completed


[TextContentBlock(text=Text(annotations=[], value="Explain deep learning to me like I'm a 5 year old. In 50 words or less."), type='text')]


[TextContentBlock(text=Text(annotations=[], value='Deep learning is like teaching a computer to think like a brain. Just

## Modifying Runs
You can only modify the metadata on runs at this time.

thread_id
(string)

Required
The ID of the thread that was run.

run_id
(string)

Required
The ID of the run to modify.

Request body
(metadata)
map

Optional
Set of 16 key-value pairs that can be attached to an object. This can be useful for storing additional information about the object in a structured format. Keys can be a maximum of 64 characters long and values can be a maxium of 512 characters long.

In [34]:
# Show our run information before modification
print(run.id)  # Print the run ID
print(run.metadata)  # Print the metadata of the run
print("\n")

# Update the run's metadata
run = client.beta.threads.runs.update(
    thread_id=run.thread_id,  # Use the thread ID from the run
    run_id=run.id,  # Use the run ID
    metadata={"user_id": "user_abc123"},  # New metadata to update
)

# Show our run information after modification
print(run.id)  # Print the run ID
print(run.metadata)  # Print the updated metadata of the run



run_MiFUmNLCvAhTQ7Z0edWTkFRu
{}


run_MiFUmNLCvAhTQ7Z0edWTkFRu
{'user_id': 'user_abc123'}


## Canceling Runs
You can cancel any run that has a state of in_progress at any time. 

thread_id
(string)

Required
The ID of the thread to which this run belongs.

run_id
(string)

Required
The ID of the run to cancel.

In [35]:
import datetime
import pytz

# Create a new run with the stream option set to True
stream_run = client.beta.threads.runs.create(
    thread_id=thread.id,  # ID of the thread to run.
    assistant_id=assistant.id,  # ID of the assistant to use.
    stream=True  # Enable streaming for the run.
)

# Variable to hold the cancelled run object
cancelled_run = None

# Continue processing the new run
for event in stream_run:
    # Check if 'status' is an attribute of the event data
    if hasattr(event.data, 'status'):
        print(event.data.id)  # Print the event ID
        print(event.data.status)  # Print the event status

        # Cancel the run if it is in progress
        if event.data.status == "in_progress":
            print("Cancelling the run.")
            cancelled_run = client.beta.threads.runs.cancel(
                thread_id=thread.id,  # ID of the thread
                run_id=event.data.id  # ID of the run to cancel
            )
    else:
        print(f"Event ID: {event.data.id} does not have a status attribute.")
        print(event.data.delta)  # Print the delta of the event data if no status attribute
    print("---------------\n")

# Retrieve the latest status of the cancelled run
cancelled_run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,  # ID of the thread
    run_id=cancelled_run.id  # ID of the cancelled run
)

# Show the run ID and when it was cancelled
print("=============== Cancelled Run Information ================\n")
print(cancelled_run.id)  # Print the run ID
print(cancelled_run.status)  # Print the status of the cancelled run
print(cancelled_run.cancelled_at)  # Print the cancellation timestamp

# Convert the created date/time in Unix format directly to Central Time and format it
formatted_date = datetime.datetime.fromtimestamp(cancelled_run.created_at, tz=pytz.utc) \
    .astimezone(pytz.timezone('America/Chicago')) \
    .strftime('%Y-%m-%d %I:%M:%S %Z')

# Print the formatted date and time
print(formatted_date)



run_nEFN1aD6RgdQafpcdRuP81jW
queued
---------------

run_nEFN1aD6RgdQafpcdRuP81jW
queued
---------------

run_nEFN1aD6RgdQafpcdRuP81jW
in_progress
Cancelling the run.
---------------

run_nEFN1aD6RgdQafpcdRuP81jW
cancelling
---------------

run_nEFN1aD6RgdQafpcdRuP81jW
cancelled
---------------

=============== Cancelled Run Information ================

run_nEFN1aD6RgdQafpcdRuP81jW
cancelled
1715888872
2024-05-16 02:47:52 CDT
